In [3]:
import pandas as pd



Match found: ('Your', 'input')
Match found: ('string', 'here')


In [6]:
import re

text = "The cat runs quickly and the dog is running too."

lemma = "run"
pattern = r'\b{}\b'.format(lemma)

matches = re.findall(pattern, text)

print(matches)

[]


In [28]:
import re
import string

text = "The cat runs quickly and the dog is running too dog."

lemma1 = "ca"
lemma2 = "do"

pattern1 = fr'\b\w*{lemma1}\w*\b'
pattern2 = fr'\b\w*{lemma2}\w*\b'

matches1 = re.findall(pattern1, text)
matches2 = re.findall(pattern2, text)

print(matches1, matches2)

masked_text = ['[FIRST_ENTITY]' if w in matches1 else '[SECOND_ENTITY]' if w in matches2 else w for w in text.translate(
    str.maketrans('', '', string.punctuation)).split()]
print(masked_text)

['cat'] ['dog', 'dog']
['The', '[FIRST_ENTITY]', 'runs', 'quickly', 'and', 'the', '[SECOND_ENTITY]', 'is', 'running', 'too', '[SECOND_ENTITY]']
